<a href="https://colab.research.google.com/github/Pusse-01/Crop-Disease-Alerting-System/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [109]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [58]:
data = pd.read_excel('/content/drive/MyDrive/plant disease detection/dataset.xlsx')

In [110]:
data.head()

,Family,temp_C,pH_val,Zone,Season
0,3,4,29,0,0
1,33,28,32,3,2
2,3,12,23,4,0
3,28,35,20,4,0
4,28,17,5,4,2


In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Crop           76 non-null     object
 1   Family         76 non-null     object
 2   Temp - C       73 non-null     object
 3   RainFall - mm  47 non-null     object
 4   PH val         72 non-null     object
 5   Zone           77 non-null     object
 6   Season         77 non-null     object
dtypes: object(7)
memory usage: 4.3+ KB


In [61]:
data.isnull().sum()

Crop              1
Family            1
Temp - C          4
RainFall - mm    30
PH val            5
Zone              0
Season            0
dtype: int64

In [62]:
data.drop(['RainFall - mm'], axis=1, inplace=True)

In [63]:
data = data.dropna()

In [64]:
data.describe()

,Crop,Family,Temp - C,PH val,Zone,Season
count,71,71,71,71,71,71
unique,71,35,51,33,12,4
top,BigOnion,Solanaceae,21-24,6.0-7.0,"dry,intermediate","yala,maha"
freq,1,10,4,13,17,60


In [65]:
data['Crop'].unique()

array(['BigOnion', 'Chili', 'RedOnion', 'Corn', 'FingerMillet', 'SoyBean',
       'BlackGram', 'Kaupi', 'Pigeon Pea', 'Potato', 'Cassava',
       'SweetPotato', 'Yams', 'GroundNut', 'Sesame', 'Mustard',
       'SunFlower', 'Cabbage', 'Radish', 'Carrot', 'Tomato', 'Capsium',
       'Brinjal', 'Okra', 'Beans', 'WingedBean', 'GreenYardLongBean',
       'RedYardLongBean', 'Pumpkin', 'Cucumber', 'SnakeGourd', 'Luffa',
       'BitterGourd', 'BeetRoot', 'AmericanOyster', 'Leeks', 'Elabatu',
       'Thibbatu', 'SpineGourd', 'Lettuce', 'IceBurg', 'KnolKhol',
       'CauliFlower', 'Gotukola', 'Mukunuwenna', 'Kathurumurunga',
       'KangKung', 'Abalone', 'PinkOyster', 'Mango', 'Pineapple',
       'Avocado', 'Banana', 'WaterMelon', 'Jamanaran', 'Duriyan', 'Papaw',
       'Guava', 'Rambutan', 'Grapes', 'Woodapple', 'Beli', 'Mangosteen',
       'Pomegranate', 'DragonFruit', 'PassionFruit', 'Anoda', 'Ugurassa',
       'StarFruit', 'Amla', 'Lawulu'], dtype=object)

In [66]:
len(data['Crop'].unique())

71

In [67]:
data['Family'].unique()

array(['Amaryllidaceae', 'Solanaceae', 'Poaceae', 'Fabaceae',
       'Pedaliaceae', 'Brassicaceae', 'Asteraceae', 'Apiaceca',
       'Malvaceca', 'Fabaceae, Genus phaseolus', 'Cucurbitaceae',
       'Chenopodiaceae', 'Agaricaceae', 'Alliaceae', 'Amaranthaceae',
       'Convolvulaceae', 'Anacardiaceae', 'Bromiliceae', 'Lauraceae',
       'Musaceae', 'Rutaceae', 'Bombacaceae', 'Caricaceae', 'Myrtaceae',
       'Sapindaceae', 'Vitaceae', 'Clusiaceae', 'Punicaceae', 'Cactaceae',
       'Passifloraciae', 'Annonaceae', 'Flacourtiaceae', 'Oxaliadaceae',
       'Euphorbiaceae', 'Sapotaceae'], dtype=object)

In [68]:
data['Season'].unique()

array(['yala', 'yala,maha', 'yala, maha', 'maha'], dtype=object)

In [69]:
data["Season"].replace({"yala": 0, "yala, maha": 2, "yala,maha": 2 ,"maha": 1}, inplace=True)
data.head()

,Crop,Family,Temp - C,PH val,Zone,Season
0,BigOnion,Amaryllidaceae,13-25,6.5-7.8,dry,0
1,Chili,Solanaceae,21-27,around 6,"dry,wet",2
2,RedOnion,Amaryllidaceae,16-21,6.0-7.0,"dry, intermediate",0
3,Corn,Poaceae,25-30,5.8-7.0,"dry,intermediate",0
4,FingerMillet,Poaceae,18-27,5.0-8.2,"dry,intermediate",2


In [70]:
data['Season'].unique()

array([0, 2, 1])

In [71]:
data['Family'].unique()
print(len(data['Family'].unique()))

35


In [72]:
data['Zone'].unique()

array(['dry', 'dry,wet', 'dry, intermediate', 'dry,intermediate',
       'wet, dry', 'dry,wet,intermediate', 'dry,intermediate,wet', 'wet',
       'intermediate,wet', 'wet,intermediate', 'wet,intermediate,dry',
       'intermediate'], dtype=object)

In [73]:
data["Zone"].replace({"dry": 0, "wet": 1, "intermediate": 2, "dry,wet": 3, "dry, intermediate": 4, "dry,intermediate": 4,
                        "wet, dry": 3, "dry,wet,intermediate": 6, "dry,intermediate,wet": 6, "intermediate,wet":5,
                        "wet,intermediate":5, "wet,intermediate,dry":6,}, inplace=True)
data.head()

,Crop,Family,Temp - C,PH val,Zone,Season
0,BigOnion,Amaryllidaceae,13-25,6.5-7.8,0,0
1,Chili,Solanaceae,21-27,around 6,3,2
2,RedOnion,Amaryllidaceae,16-21,6.0-7.0,4,0
3,Corn,Poaceae,25-30,5.8-7.0,4,0
4,FingerMillet,Poaceae,18-27,5.0-8.2,4,2


In [74]:
data['Zone'].unique()

array([0, 3, 4, 6, 1, 5, 2])

In [75]:
data['PH val'].unique()

array(['6.5-7.8', 'around 6', '6.0-7.0', '5.8-7.0', '5.0-8.2', '6.0-6.5',
       '5.0-8.5', '5.5-6.6', '5.5-5.6', '5.3-5.9', '5.4-5.9', '6.5-7.0',
       '5.5-8.0', '6.0-7.5', '6.5-8.5', '5.5-7.5', '5.5-6.8', '5.5-5.8',
       '6.0-6.8', '5.5-6.5', '4.3-7.5', '5.6-7.0', '6.3-7.3', '5.0-6.0',
       '5.6-5.7', '5.5-7.0', '5.0-6.5', '4.5-7.0', '6.5-7.5', '6.0-9.0',
       '5.0-10.0', '5.5-7.2', '6.5-9.5'], dtype=object)

In [76]:
data['Temp - C'].unique()

array(['13-25', '21-27', '16-21', '25-30', '18-27', '26-32', '20-35',
       '15-20', '<24', '25-29', '20-40', '29-30', '14-30', '>20', '15-18',
       '10.0-15.0', '21-24', '23-32', '<30', '18-32', '20-26', '23-29',
       '20-32', '24-31', '18-25', '17-19', '13-23', '<20', '15-25',
       '18-20', '18-30', '>27', '18-24', '27-30', '24-32', '26-30',
       '15-38', '12.0-37.0', '28-35', '20-28', '25-32', '25-37', '25-35',
       '25-28', '5.0-38', '20-30', '20-27', '13-29', '12.0-18.0', '<46',
       '14-36'], dtype=object)

In [82]:
data['Family'] = data.Family.astype("category").cat.codes

In [84]:
data.rename(columns = {'Temp - C':'temp_C', 'PH val':'pH_val'}, inplace = True)

In [86]:
data['temp_C'] = data.temp_C.astype("category").cat.codes
data['pH_val'] = data.pH_val.astype("category").cat.codes

In [88]:
data.drop(['Crop'], axis=1, inplace=True)

In [89]:
data.head()

,Family,temp_C,pH_val,Zone,Season
0,3,4,29,0,0
1,33,28,32,3,2
2,3,12,23,4,0
3,28,35,20,4,0
4,28,17,5,4,2


In [90]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 0 to 75
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Family  71 non-null     int8 
 1   temp_C  71 non-null     int8 
 2   pH_val  71 non-null     int8 
 3   Zone    71 non-null     int64
 4   Season  71 non-null     int64
dtypes: int64(2), int8(3)
memory usage: 1.9 KB


In [91]:
import plotly.express as px
fig = px.scatter_matrix(data,
width=1200, height=1600)
fig.show()

In [92]:
X=data
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)
inertia = []
for i in range(1,11):
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
fig = go.Figure(data=go.Scatter(x=np.arange(1,11),y=inertia))
fig.update_layout(title="Inertia vs Cluster Number",xaxis=dict(range=[0,11],title="Cluster Number"),
                  yaxis={'title':'Inertia'},
                 annotations=[
        dict(
            x=3,
            y=inertia[2],
            xref="x",
            yref="y",
            text="Elbow!",
            showarrow=True,
            arrowhead=7,
            ax=20,
            ay=-40
        )
    ])

In [97]:
kmeans = KMeans(
        n_clusters=3, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
kmeans.fit(X)
clusters=pd.DataFrame(X,columns=data.columns)
clusters['label']=kmeans.labels_
polar=clusters.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
fig4 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
fig4.show()

In [98]:
pie=clusters.groupby('label').size().reset_index()
pie.columns=['label','value']
px.pie(pie,values='value',names='label',color=['blue','red','green'])

In [99]:
cols = data.columns

In [100]:
ms = MinMaxScaler()
X = ms.fit_transform(data)

In [101]:
X = pd.DataFrame(X, columns=[cols])

In [102]:
X.head()

,Family,temp_C,pH_val,Zone,Season
0,0.088235,0.08,0.90625,0.000000,0.0
1,0.970588,0.56,1.00000,0.500000,1.0
2,0.088235,0.24,0.71875,0.666667,0.0
3,0.823529,0.70,0.62500,0.666667,0.0
4,0.823529,0.34,0.15625,0.666667,1.0


In [106]:
kmeans.cluster_centers_

array([[0.34338235, 0.555     , 0.58515625, 0.7625    , 0.9875    ],
       [0.86898396, 0.48909091, 0.36221591, 0.8030303 , 1.        ],
       [0.52941176, 0.43111111, 0.61805556, 0.44444444, 0.05555556]])

In [107]:
kmeans.labels_

array([2, 1, 2, 2, 1, 0, 0, 2, 2, 2, 1, 1, 1, 0, 1, 0, 0, 2, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 2, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 1], dtype=int32)

In [108]:
sample_test = np.array([0.088235,0.08, 0.90625, 0.000000,0.0])
second_test=sample_test.reshape(1, -1)
kmeans.predict(second_test)

array([2], dtype=int32)